In [8]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [87]:
import pandas as pd
import os
import nltk
import pandas as pd
import numpy as np
import string
import warnings
import re
warnings.filterwarnings('ignore')


In [103]:
import requests
from bs4 import BeautifulSoup
import openpyxl

# Load the Excel file
input_workbook = openpyxl.load_workbook('Input.xlsx')
input_sheet = input_workbook.active

# Iterate through each row in the Excel file
for row in input_sheet.iter_rows(min_row=2, values_only=True):
    url_id, url = row
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        article_title = soup.find('h1').text

        # Check for different div classes
        article_div = soup.find('div', class_='td-post-content tagdiv-type') or soup.find('div', class_="td_block_wrap tdb_single_content tdi_130 td-pb-border-top td_block_template_1 td-post-content tagdiv-type") or soup.find('p')

        # Extract article text
        article_text = article_title + '\n\n' + article_div.get_text()

        # Save the article text to a text file in Google Drive
        file_name = '/content/gdrive/MyDrive/Blackcoffer/Text Directory/' + str(url_id) + '.txt'
        with open(file_name, 'w') as file:
          file.write(article_text)

    else:
        print(f"Failed to retrieve content from {url_id}")

# Close the input workbook
input_workbook.close()


Failed to retrieve content from 11668.0
Failed to retrieve content from 17671.4


In [104]:
# The code defines a function called `generate_stopwords()` that reads multiple text files and extracts the stopwords from them.
def generate_stopwords():
    parent_path = "/content/gdrive/MyDrive/Blackcoffer/StopWords/StopWords_"
    child_paths = ['Auditor', 'Currencies', 'DatesandNumbers','Generic', 'GenericLong', 'Geographic', 'Names']
    words = []
    for child in child_paths:
        text = ""
        with open(f"{parent_path}{child}.txt", 'r', encoding = "utf-8", errors = "ignore") as f:
            words.extend([word for line in f for word in line.split() if word not in ['\n', '|']])
    return words

In [105]:
# The code is generating a master dictionary by reading text files containing positive and negative words.
def generate_MasterDict(stopwords):
    parent_path = "/content/gdrive/MyDrive/Blackcoffer/MasterDictionary/"
    child_paths = ['positive-words', 'negative-words']
    words = []
    for child in child_paths:
        text = ""
        with open(f"{parent_path}{child}.txt", 'r', encoding = "utf-8", errors = "ignore") as f:
            words.append([word for line in f for word in line.split() if word not in stopwords+['\n', '|']])
    return words

In [106]:
def generate_content(url_id):
    text = ""
    with open(f"/content/gdrive/MyDrive/Blackcoffer/Text Directory/{url_id}.txt", 'r', encoding = "utf-8", errors = "ignore") as f:
        text += f.read()
    sent = nltk.sent_tokenize(text)
    text = text.translate(str.maketrans('','',string.punctuation))
    words = nltk.word_tokenize(text)
    words = [w for w in words if w not in stopwords]
    return sent, words

In [107]:
# The `AvgSyllable` function is a Python function that calculates various metrics related to syllables and complexity in a given list of words.

def AvgSyllable(words):
    plurals = "aeiouAEIOU"
    total_syallable = 0
    complex_words = 0
    for w in words:
        if not w.endswith(('es', 'ed')):
            syallable = sum([1 for c in w if c in plurals])
            if(syallable > 2): complex_words += 1
            total_syallable += syallable
    syllable_per_word = (total_syallable/len(words))
    complex_percent = (complex_words)/len(words)
    fog_index = 0.4 * (syllable_per_word + complex_percent)
    return syllable_per_word,  complex_words, complex_percent, fog_index

In [108]:
# The code is defining a regular expression pattern `pronounRegex` that matches common pronouns such as "I", "we", "my", "ours", and "us".
# This function calculating the count of pronouns in the given sentence.
def Calculate_pronouns(sent):
    pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
    pronouns = []
    for s in sent:
        if s: pronouns.extend(re.findall(pronounRegex, s))
    pronouns_count = sum([1 for w in pronouns])
    return pronouns_count

In [109]:
stopwords = generate_stopwords()
MasterWords = generate_MasterDict(stopwords)
Pos_words = MasterWords[0]
Neg_words = MasterWords[1]
df = pd.read_excel("/content/gdrive/MyDrive/Blackcoffer/Output Data Structure.xlsx")
df.index = df.URL_ID
df.drop(["URL_ID"],axis = 1, inplace = True)

In [110]:
df.head()

,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
URL_ID,,,,,,,,,,,,,,
123.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4321.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
432.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
df.columns

Index(['URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
       'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
       'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'],
      dtype='object')

In [112]:
for url_id in df.index:
    if url_id not in [11668, 17671.4 ]:

        sent, words = generate_content(url_id)
        pos_score = sum([1 for w in words if w in Pos_words])
        neg_score = sum([1 for w in words if w in Neg_words])
        Syllable_per_word , complex_count, complex_percentage, fog_index = AvgSyllable(words)
        pronouns_count = Calculate_pronouns(sent)


        # The code snippet is assigning values to various columns in a DataFrame `df` based
        # on the calculations and assignments performed in the above code.

        df["POSITIVE SCORE"][url_id] =  pos_score
        df["NEGATIVE SCORE"][url_id] = neg_score
        df["POLARITY SCORE"][url_id] = (pos_score - neg_score)/(pos_score + neg_score + 0.000001)
        df['SUBJECTIVITY SCORE'][url_id] = (pos_score + neg_score)/(len(words)+0.000001)
        df['AVG SENTENCE LENGTH'][url_id] = len(words)/len(sent)
        df['PERCENTAGE OF COMPLEX WORDS'][url_id] = complex_percentage
        df['FOG INDEX'][url_id] = fog_index
        df['AVG NUMBER OF WORDS PER SENTENCE'][url_id] = len(words)/len(sent)
        df['COMPLEX WORD COUNT'][url_id] = complex_count
        df["WORD COUNT"][url_id] = len(words)
        df['SYLLABLE PER WORD'][url_id] = Syllable_per_word
        df['PERSONAL PRONOUNS'][url_id] = pronouns_count
        df['AVG WORD LENGTH'][url_id] = sum([len(w) for w in words])/len(words)

In [118]:
df.isnull().sum()

URL                                 0
POSITIVE SCORE                      2
NEGATIVE SCORE                      2
POLARITY SCORE                      2
SUBJECTIVITY SCORE                  2
AVG SENTENCE LENGTH                 2
PERCENTAGE OF COMPLEX WORDS         2
FOG INDEX                           2
AVG NUMBER OF WORDS PER SENTENCE    2
COMPLEX WORD COUNT                  2
WORD COUNT                          2
SYLLABLE PER WORD                   2
PERSONAL PRONOUNS                   2
AVG WORD LENGTH                     2
dtype: int64

In [119]:
df = df.dropna()

In [122]:
df.isnull().sum()

URL                                 0
POSITIVE SCORE                      0
NEGATIVE SCORE                      0
POLARITY SCORE                      0
SUBJECTIVITY SCORE                  0
AVG SENTENCE LENGTH                 0
PERCENTAGE OF COMPLEX WORDS         0
FOG INDEX                           0
AVG NUMBER OF WORDS PER SENTENCE    0
COMPLEX WORD COUNT                  0
WORD COUNT                          0
SYLLABLE PER WORD                   0
PERSONAL PRONOUNS                   0
AVG WORD LENGTH                     0
dtype: int64

In [123]:
df.to_excel("Output Data.xlsx")

In [124]:
df.to_csv("Output_Data.csv")